In [19]:
from pathlib import Path

result_dir = Path("/home/u5u/kdeng.u5u/spatial-reasoning-of-LMs/result/basic-rela-cls")

In [20]:
import jsonlines

data = []
for model_dir in result_dir.iterdir():
    if model_dir.is_dir():
        inf_file = model_dir / "inference.jsonl"
        with jsonlines.open(inf_file, mode="r") as reader:
            for row in reader:
                data.append(row)

In [21]:
import pandas as pd

df = pd.DataFrame(data)

In [22]:
# define order
desired_order = [
    'instructblip-vicuna-7b',
    "SpaceQwen2.5-VL-3B-Instruct",
    "Idefics3-8B-Llama3",
    "llava-onevision-qwen2-7b-ov-hf",
    "Qwen2.5-VL-7B-Instruct",
    "Qwen2.5-VL-32B-Instruct",
    "Qwen2.5-VL-72B-Instruct",
    "gpt-4o",
]

# change row into cate data type and sort
df["vlm_id"] = df["vlm_id"].apply(lambda id: id.split("/")[-1])
df['vlm_id'] = pd.Categorical(df['vlm_id'], categories=desired_order, ordered=True)



In [23]:
from sklearn.metrics import accuracy_score, f1_score

def acc_f1(group):
    acc = accuracy_score(group["cor_idx_shuf"], group["pred"])
    f1 = f1_score(group["cor_idx_shuf"], group["pred"], average="weighted")
    return pd.Series({"Acc.": acc, "F1": f1})

In [24]:
df["cor_rela"].value_counts() / 8

cor_rela
left      11.0
right     10.0
behind     6.0
front      6.0
above      4.0
below      4.0
top        2.0
Name: count, dtype: float64

In [28]:
table = (df.groupby(["vlm_id"]).apply(acc_f1)).round(2)
table

/local/user/1483801110/ipykernel_40359/2154199529.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  table = (df.groupby(["vlm_id"]).apply(acc_f1)).round(2)
/local/user/1483801110/ipykernel_40359/2154199529.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  table = (df.groupby(["vlm_id"]).apply(acc_f1)).round(2)


,Acc.,F1
vlm_id,,
instructblip-vicuna-7b,0.21,0.20
SpaceQwen2.5-VL-3B-Instruct,0.30,0.22
Idefics3-8B-Llama3,0.40,0.38
llava-onevision-qwen2-7b-ov-hf,0.56,0.56
Qwen2.5-VL-7B-Instruct,0.30,0.28
Qwen2.5-VL-32B-Instruct,0.67,0.68
Qwen2.5-VL-72B-Instruct,0.70,0.69
gpt-4o,0.63,0.63


In [26]:
# rename the columns


In [29]:
print(table.to_latex(
    escape=False, 
    index=True, 
    float_format="%.2f",
    column_format="lcc",
    caption="Performance of basic relation classification task.",
    label="tab:basic-rela-cls",
    position="tb",
))

\begin{table}[tb]
\caption{Performance of basic relation classification task.}
\label{tab:basic-rela-cls}
\begin{tabular}{lcc}
\toprule
 & Acc. & F1 \\
vlm_id &  &  \\
\midrule
instructblip-vicuna-7b & 0.21 & 0.20 \\
SpaceQwen2.5-VL-3B-Instruct & 0.30 & 0.22 \\
Idefics3-8B-Llama3 & 0.40 & 0.38 \\
llava-onevision-qwen2-7b-ov-hf & 0.56 & 0.56 \\
Qwen2.5-VL-7B-Instruct & 0.30 & 0.28 \\
Qwen2.5-VL-32B-Instruct & 0.67 & 0.68 \\
Qwen2.5-VL-72B-Instruct & 0.70 & 0.69 \\
gpt-4o & 0.63 & 0.63 \\
\bottomrule
\end{tabular}
\end{table}

